# Toxicity in Wikipedia Comments

This is a parallel work to any work on the wikipedia toxicity data on the same
topic.  This data has not been cleaned yet, and has not had multiple categories introduced yet.  However it is presented free from bias, for people to play with.  

In [2]:
import pandas as pd
import nltk as nltk
import numpy as np

In [7]:
ls -lh data

total 128M
-rw-rw-r-- 1 jonathan jonathan  18M Jan 14 10:28 boardgame-frequent-user-comments.csv
-rw-rw-r-- 1 jonathan jonathan  75M Jan 14 10:27 toxicity_annotated_comments.tsv
-rw-rw-r-- 1 jonathan jonathan 692K Jan  7 14:27 toxicity_annotated_comments_unanimous.tsv
-rw-rw-r-- 1 jonathan jonathan  35M Jan 14 10:27 toxicity_annotations.tsv
-rw-rw-r-- 1 jonathan jonathan 775K Jan  7 14:27 toxicity_annotations_unanimous.tsv
-rw-rw-r-- 1 jonathan jonathan  93K Jan  7 14:27 toxicity_worker_demographics.tsv


In [71]:
?pd.read_csv

In [3]:
df_com = pd.read_csv('data/toxicity_annotated_comments_unanimous.tsv',sep='\t')
df_rate = pd.read_csv('data/toxicity_annotations_unanimous.tsv',sep='\t')
#make rev_id an integer
df_com['rev_id']=df_com['rev_id'].astype(int)
df_rate['rev_id']=df_rate['rev_id'].astype(int)

#reindex 
#df_com.index=df_com['rev_id']
#df_rate.index=df_rate['rev_id']
print(df_com.shape, df_rate.shape)

(3582, 7) (35152, 4)


In [137]:
#make a new column in df_com with array of worker_ids, and toxicity
df_com['scores']=None

#since 'rev_id' is sorted, can take first difference, and find where
#there are changes.  Those set the boundaries for changes.
#Need to also append final index.
change_indices=df_rate.index[df_rate['rev_id'].diff()!=0].values
change_indices=np.append(change_indices,len(df_rate))

for i in range(len(change_indices)-1):
    ind0 = change_indices[i]
    ind1 = change_indices[i+1]
    d0=df_rate.iloc[ind0:ind1]
    scores=d0[['worker_id','toxicity_score']]
    #pass it a list so it can be set as an entry.
    #accessing later will require score[0] idiocy to get at list.
    df_com.loc[i,'scores']=[scores.values]


In [158]:
def score_mean(score_list):
    """score_mean
    Compute mean of toxicity scores for input array.
    Array is first (and only) element in the input list.
    Compute mean running down the rows.  Could be updated to include weighted sum of weights
    """
    s_arr=score_list[0]
    s = np.mean(s_arr[:,1])
    return s

score_mean(s)

-1.0

In [161]:
df_com['mean_toxic']=df_com['scores'].apply(score_mean)
df_com['toxic']=df_com['mean_toxic']!=0

In [162]:
df_com['toxic'].sum()

395

In [74]:
df_rate.groupby('rev_id')['toxicity_score'].mean()

rev_id
527004       0.0
603474       0.0
808576       0.0
986363       0.0
1433843      0.0
1988528      0.0
1988988      0.0
2078042      0.0
2747887      0.0
2850252      0.0
3072376      0.0
3833612      0.0
4402645      0.0
5501854      0.0
5996506      0.0
6117068      0.0
6569572      0.0
7005403      0.0
7278042      0.0
7482390      0.0
8078390      0.0
8190858      0.0
8355464      0.0
8384537      1.0
8410988      0.0
8470953      0.0
8486458      0.0
8898541      0.0
9125825      0.0
9180326      0.0
            ... 
693072891    0.0
693172874    0.0
693258130    0.0
693591656    0.0
693603090    0.0
693758023    0.0
694377826    0.0
694418527    0.0
694524243    0.0
695346026    0.0
695411224    0.0
695437195    0.0
695519219    0.0
695673624    0.0
695755011    0.0
696040084    0.0
696130208    0.0
696215805    0.0
696223206    0.0
696721125    0.0
696792920    0.0
696803203    0.0
696827903    0.0
696920504   -1.0
697089785    0.0
697514711    0.0
697541689    0.0
6977966

In [ ]:
#can combine the dataframes together by extracting all reviewer ids, and scores for each 

Looking at the comments data, we'll need to clean the data quite a bit (lots of newlines, weird characters).

My rough plan is to build up a lexicon, tokenize that data, and try to build a Naive Bayes model.  (Maybe later a Recurrent Neural network model?)

Other Analysis possibilities:
* Support Vector Machine
    - the other big architecture, less popular now?
* Recurrent Neural Network
    - Build up word embeddings (word2vec), or just use the pretrained ones.
* Naive Bayes
    - can find most important words in spam
    - simple, easy to understand baseline.
* Latent Factor Analysis 
    - maybe useful prelude for building up embeddings. 
    
Cleaning:
* Tokenize (convert words to indices)
* Clean data : How to remove newlines (search/replace: NEWLINE with '')
* Stemming words
* Balancing data set
* Match up comments, and review scores


* Search for gibberish words (make a new "feature" for badly spelled comments)

In [19]:
#borrowing from http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()
X_train_counts=count_vect.fit_transform(df_com['comment'])

In [23]:
X_train_counts.shape

(3582, 11554)